<h3 style = "background-color: #000033;
             padding: 15px;
             font: bold 42px arial;
             color: #ccebff;
             border: 2px #e6e6ff;
             border-radius: 8px">
 AMAZON BUSINESS AND SALES ANALYTICS </h3>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

: 

In [ ]:
data=pd.read_csv('/kaggle/input/amazon-products-dataset/Amazon-Products.csv')

In [ ]:
data.sample(10)

In [ ]:
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

# Data Cleaning

In [ ]:
data.shape

In [ ]:
#checking for  duplicates
data.duplicated().sum()

In [ ]:
data.drop_duplicates(inplace=True)

In [ ]:
data.shape

In [ ]:
data=data.loc[:,['name','main_category','sub_category','ratings','no_of_ratings','discount_price','actual_price']]

In [ ]:
data.columns

In [ ]:
data.info()

In [ ]:

# Apply strip to all cells that are strings
data = data.map(lambda x: x.strip() if isinstance(x, str) else x)



In [ ]:
data.shape

In [ ]:
data.isnull().sum()

In [ ]:
#calculating %missing data

percent_miss=[]
for col in data.columns:
    percentage=(data[col].isnull().sum()/data.shape[0])*100
    print('{}-  {}%'.format(col,np.round(percentage,4)))

In [ ]:
# since actual_price column has less than 5% null values we can apply complete case analysis

In [ ]:
#columns with missing data <5%
cols = [var for var in data.columns if data[var].isnull().mean()*100 < 5 and data[var].isnull().mean()*100 > 0]
cols

In [ ]:
data.dropna(subset=cols, inplace=True)

In [ ]:
data.shape

In [ ]:
data['ratings'].shape

In [ ]:
data.dtypes

In [ ]:
# clearly apart from name, main_catefory, sub_category, rest all columns should be numerical

# Preprocessing all columns


  <div style="background-color: #e7f3fe; 
            padding: 20px; 
            font: bold 18px Arial; 
            color: #31708f; 
            border: 2px solid #bce8f1; 
            border-radius: 8px;">
Preprocessing rating column
</div>

In [ ]:
data['ratings'].value_counts()

In [ ]:
data['ratings'].unique()

In [ ]:
pd.set_option('display.max_rows',None) 
data['no_of_ratings'].unique()

In [ ]:
# we can clearly see this rating column needs to be cleaned
# coulumn having ratings Get, Free, ₹99 ,₹70, ₹2.99,₹68.99,₹65, ₹100  will be filled with rating nan   

In [ ]:
data['ratings']=data['ratings'].replace(to_replace=['Get','FREE','₹99','₹70','₹2.99','₹68.99','₹65','₹100'],value=np.nan)

In [ ]:
# remove # and run the following command
#data['ratings'].value_counts()

In [ ]:
data['ratings']=data['ratings'].astype('float')

In [ ]:
data['ratings']=data['ratings'].round(2)


 
 <div style="background-color: #e7f3fe; 
            padding: 20px; 
            font: bold 18px Arial; 
            color: #31708f; 
            border: 2px solid #bce8f1; 
            border-radius: 8px;">
Preprocessing main_category column
</div>

In [ ]:
data['main_category'].value_counts()


 
  <div style="background-color: #e7f3fe; 
            padding: 20px; 
            font: bold 18px Arial; 
            color: #31708f; 
            border: 2px solid #bce8f1; 
            border-radius: 8px;">
Preprocessing sub_category column
</div>

In [ ]:
data['sub_category'].value_counts()

In [ ]:
# Adjust display options
pd.set_option('display.max_rows', None)  # Show all rows for value_counts()

# View all value counts for 'no_of_ratings'
print(data['sub_category'].value_counts())

In [ ]:
# Clean and normalize the subcategory column
data['sub_category'] = data['sub_category'].str.strip().str.lower()

In [ ]:
data['sub_category'].nunique()

In [ ]:
# we have so many subcategory so we are grouping this

In [ ]:
counts=data['sub_category'].value_counts()
uncommon_category=counts[counts<1300].index
data['sub_category'] = np.where(data['sub_category'].isin(uncommon_category), 'uncommon mix category', data['sub_category'])


In [ ]:
data['sub_category'] = np.where(data['sub_category'].str.contains('shoe|sandals|ballerinas|foot'), 'footwear', data['sub_category'])
data['sub_category'] = np.where(data['sub_category'].str.contains('bag|backpack|sack'), 'bags', data['sub_category'])
data['sub_category'] = np.where(data['sub_category'].str.contains('nightwear|lingerie|innerwear'), 'Innerwear', data['sub_category'])
data['sub_category'] = np.where(data['sub_category'].str.contains('nightwear|camping|strength|lingerie|running|sports|football|cricket|badminton|fitness|diet|cycling|yoga'), 'Sports', data['sub_category'])
data['sub_category'] = np.where(data['sub_category'].str.contains('toy|pram'), 'toys', data['sub_category'])
data['sub_category'] = np.where(data['sub_category'].str.contains('speakers|headphones|appliances|televisions|system|electronics|camera|refrigerators|washing|machines|theater|air conditioners'), 'Electronics', data['sub_category'])
data['sub_category'] = np.where(data['sub_category'].str.contains('home|kitchen|room|dine|house|indoor|furniture|sewing'), 'Home and Kitchen', data['sub_category'])
data['sub_category'] = np.where(data['sub_category'].str.contains('kid|baby|diaper'), 'Kids', data['sub_category'])
data['sub_category'] = np.where(data['sub_category'].str.contains('beauty|make-up|personal care'), 'Cosmetics', data['sub_category'])

In [ ]:
data['sub_category'].value_counts()

In [ ]:
data['sub_category'].nunique()

In [ ]:
data.dtypes

# Handling Outliers

In [ ]:
def handle_outlier(data,column):
    sns.boxplot(data[column])
    plt.title("outlier detection")
    plt.show()
    q1=data[column].quantile(0.25)
    q3=data[column].quantile(0.75)
    iqr=q3-q1
    upper_limit= q3+1.5*iqr
    lower_limit=q1-1.5*iqr

    data[column]=np.where(
        data[column]>upper_limit, upper_limit, np.where(data[column]<lower_limit,lower_limit,data[column])
    )
    
    sns.boxplot(data[column])
    plt.title("outliers removed") 
    plt.show()

In [ ]:
handle_outlier(data,'ratings')

In [ ]:
#handle_outlier(data,'no_of_ratings')

In [ ]:
#handle_outlier(data,'discount_price')

##### Clearly 75% and max value for no_of_ratings  and discount_price has too much difference so we should not remove our outliers for these features.
##### Moreover, outliers of these columns are important from analysis point of view. 

# Descriptive Statistics 


<h3 style = "background-color: #111;
             padding: 15px;
             font: bold 22px arial;
             color: lightgreen;
             border: 2px solid lime;
             border-radius: 8px">
 1) Descriptive Statistics 📉📊📈📘</h3>

In [ ]:
np.round(data.describe(),2)

# Discount Analysis


<h3 style = "background-color: #111;
             padding: 15px;
             font: bold 22px arial;
             color: lightgreen;
             border: 2px solid lime;
             border-radius: 8px">
 2) Discount Analysis 🛒</h3>

#### 2a) Overall Average discount

In [ ]:
average_discount=np.mean(data['actual_price']-data['discount_price'])

In [ ]:
print("Average discount given is ",np.round(average_discount,3))

In [ ]:
# Calculate the discount amount and percentage
data['discount_amount'] = data['actual_price'] - data['discount_price']
data['discount_percentage'] = (data['discount_amount'] / data['actual_price']) * 100

#### 2b) Average discount across categories

In [ ]:
data.loc[data['discount_percentage']==-np.inf]

In [ ]:
#note that since actual_price is zero.
#discount_percentage is getting infinity

In [ ]:
# Handle infinity
data['discount_percentage'] = data['discount_percentage'].replace([np.inf, -np.inf], 0)
# discount_percentage below 0 are set to 0. 
data['discount_percentage'] = data['discount_percentage'].clip(lower=0)


In [ ]:
overall_average_discount_percentage=np.mean(data['discount_percentage'] )
print("Overall Average Discount Percentage is ",np.round(overall_average_discount_percentage,2))

In [ ]:
discount_analysis_main_category = data.groupby(['main_category']).agg({
    'discount_amount': 'mean',
    'discount_percentage': 'mean'
}).round(2)
discount_analysis_main_category

In [ ]:
# Reset index for easier plotting
discount_analysis_main_category_reset = discount_analysis_main_category.reset_index()

# Plotting using seaborn barplot
plt.figure(figsize=(12, 6))
bars=sns.barplot(y=discount_analysis_main_category_reset['discount_percentage'], x=discount_analysis_main_category_reset['main_category'] ,hue=discount_analysis_main_category_reset['main_category'] 
            ,palette='summer')

# Add labels to the bars
for bar in bars.containers:
    bars.bar_label(bar, fmt='%.2f%%', fontsize=11.2, color='black')


plt.title('Discount distribution across Main Category',fontsize=15)
plt.xlabel('Mean Discount Percentage',fontsize=10)
plt.ylabel('Main category',fontsize=10)
plt.xticks(rotation=90,fontsize=10)
plt.tight_layout()

plt.show()

##### Minimum discount is given to "home, kitchen pets category" ,namely 20.09%

##### Maximum discount is given to women's clothing, namely 57.98%

In [ ]:
discount_analysis = data.groupby(['main_category', 'sub_category']).agg({
    'discount_amount': 'mean',
    'discount_percentage': 'mean'
}).round(2).sort_values(by=['discount_percentage'])

In [ ]:
discount_analysis

>
##### minimum discount of 20.09% is given to "home, kitchen, pets" section, specifically to uncommon category

>
##### maximum discount of 64.76% is given to "accessories" section specifically to "fashion & silver jewellery"

#### 2c) Correlation between 'discount_price' and 'actual_price'

In [ ]:
# Calculate the correlation between 'discount_price' and 'actual_price'
correlation = data['discount_price'].corr(data['actual_price'])

# Print the result
print(f"The correlation between discount_price and actual_price is: {correlation}")

In [ ]:
corr_mat=data[['discount_price' ,'actual_price']].corr()
sns.heatmap(corr_mat,annot=True, cmap='summer', fmt='.3f', square=True)

>
##### This shows that correlation between discount_price and actual_price is approximately zero.
##### Thus discount_price and actual_price are independent of each other

# Ratings Analysis



<h3 style = "background-color: #111;
             padding: 15px;
             font: bold 22px arial;
             color: lightgreen;
             border: 2px solid lime;
             border-radius: 8px">
 3) Ratings Analysis⭐️⭐️⭐️⭐️⭐️ </h3>

#### a) Calculate the average rating for each main_category .

In [ ]:
avg_ratings=np.round(data['ratings'].mean(),2)
print("Overal Average Rating is",avg_ratings)

In [ ]:
main_category_grouped=data.groupby('main_category')
main_category_avg_rating=np.round(main_category_grouped['ratings'].mean(),2)
main_category_avg_rating.sort_values(ascending=True)

#### b) Identify category with the highest and lowest ratings.

In [ ]:
# Find the highest and least rated categories

highest_rated_category = main_category_avg_rating.idxmax()
highest_rating = main_category_avg_rating.max()

least_rated_category = main_category_avg_rating.idxmin()
least_rating = main_category_avg_rating.min()

# Print the results
print(f"Highest rated category: '{highest_rated_category}' with a rating of {highest_rating}")
print(f"Least rated category: '{least_rated_category}' with a rating of {least_rating}")

In [ ]:
# Plotting
plt.figure(figsize=(12,6))
bars=sns.barplot(x=main_category_avg_rating.index,y=main_category_avg_rating.values,color='pink')
plt.bar_label(bars.containers[0], labels=main_category_avg_rating,fontsize=11.5,color='black')
plt.xticks(rotation=90,fontsize=10)
plt.title("Avg Rating across Main Category",fontsize=15)
plt.xlabel("Main Category  --->",fontsize=10)
plt.ylabel("Avg Rating  --->",fontsize=10)
plt.ylim(0,5)
plt.show()

>

##### least rated category- "home, kitchen, pets" 

>
#####  Clearly,  highest rated category-"grocery & gourmet foods"
        

#### c) Calculate the average rating for each main_category and sub_category 

In [ ]:
# Group by 'main_category' and 'sub_category' and calculate the mean ratings
grouped = data.groupby(['main_category', 'sub_category'])
avg_ratings = np.round(grouped['ratings'].mean(), 2)
# Print the result
print(avg_ratings)

In [ ]:
sub_category_grouped=data.groupby('sub_category')
sub_category_avg_rating=np.round(sub_category_grouped['ratings'].mean(),2)
sub_category_avg_rating=sub_category_avg_rating.sort_values(ascending=True)
sub_category_avg_rating

In [ ]:
plt.figure(figsize=(12,7))
bars=sub_category_avg_rating.sort_values().plot(kind='barh',figsize=(10,8),title="RATINGS ACROSS DIFFERENT SUBCATEGORIES",color='#9B59B6',alpha=0.7)
plt.bar_label(bars.containers[0], labels=sub_category_avg_rating, padding=2,fontsize=13)
plt.title("Ratings Across SubCategory -->",fontsize=15)
plt.ylabel("Sub Category", fontsize=10)
plt.xlabel("Avg Rating -->", fontsize=10)
plt.yticks(fontsize=10)
plt.xlim(0,5)
plt.show()

##### "Kids" is least rated subcategory and "Sports" is highly rated

##### It is important to note that every category has got avg rating >3.5

#### 3 d) the relationship between ratings and no._of_ratings.

In [ ]:
correlation = data['ratings'].corr(data['no_of_ratings'])
print(f"The correlation between ratings and no._of_ratings is: {correlation:.2f}")

##### Correlation of 0.03 suggests that knowing the rating of a product does not give useful information about the number of ratings it has received.

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(data['ratings'], data['no_of_ratings'],c='#2ECC71')
plt.title('Scatter Plot of Ratings vs Number of Ratings')
plt.xlabel('Ratings')
plt.ylabel('Number of Ratings')
plt.show()

# Product analysis


<h3 style = "background-color: #111;
             padding: 15px;
             font: bold 22px arial;
             color: lightgreen;
             border: 2px solid lime;
             border-radius: 8px">
 4) Product Analysis 🚚🎒🔍 </h3>

#### 4 a) Number Of Products in different Categories

In [ ]:
data_group = data.groupby(['main_category']).agg({'name':'count'}).reset_index()
data_group.columns = ['main_category', 'No_of_products']

# Print the result
print(data_group)


In [ ]:
plt.figure(figsize=(12,6))
bars=plt.bar(data_group['main_category'],data_group['No_of_products'],color='skyblue',alpha=1)
plt.bar_label(bars, label=data_group['No_of_products'],label_type='edge',fontsize=11,color='black')
plt.xticks(rotation=90,fontsize=10)
plt.yticks(fontsize=10)
plt.xlabel("Main Categories -->", fontsize=10)
plt.ylabel("Product Count -->" ,fontsize=10)
plt.title("NUMBER OF PRODUCTS IN DIFFERENT CATEGORIES",fontsize=15)
plt.show()

#### 4b) Products with the highest number of ratings (TOP 10 PRODUCTS)

In [ ]:
data_gr=data.groupby(['name']).agg({'no_of_ratings': 'sum'}).sort_values(by='no_of_ratings',ascending=False)

# Assign ranks, ensuring that products with the same number of ratings get the same rank
data_gr['rank'] = data_gr['no_of_ratings'].rank(method='dense', ascending=False)

top_n = 10
top_products = data_gr[data_gr['rank'] <= top_n]
top_products.columns=['no_of_ratings','rank']

In [ ]:
print(top_products)

In [ ]:
# plot top 3 products

top_3_products = data_gr[data_gr['rank'] <= 3]

# Plotting the top 3 products
plt.figure(figsize=(12, 6))
sns.barplot(x=top_3_products.index, y='no_of_ratings', data=top_3_products,width=0.4)
plt.title('Top 3 Products by Number of Ratings',fontsize=15)
plt.xlabel('Product Name',fontsize=10)
plt.ylabel('Number of Ratings',fontsize=10)
                                                                                   
plt.xticks(rotation=90,fontsize=10)

plt.show()

# Correlation Analysis


<h3 style = "background-color: #111;
             padding: 15px;
             font: bold 22px arial;
             color: lightgreen;
             border: 2px solid lime;
             border-radius: 8px">
 5) Correlation Analysis✍️ </h3>

In [ ]:
correlation_matrix = data[['ratings', 'no_of_ratings', 'discount_price', 'actual_price']].corr()
print(correlation_matrix)

In [ ]:
# Plot heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='cividis', fmt='.3f', square=True)
plt.title('Correlation Matrix Heatmap',fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15,rotation=360)
plt.show()

##### Clearly these values are close to zero. Hence, there is no linear relationship between variables.

# Revenue Analysis


<h3 style = "background-color: #111;
             padding: 15px;
             font: bold 22px arial;
             color: lightgreen;
             border: 2px solid lime;
             border-radius: 8px">
  6)Revenue Analysis 💰💸🧾️ </h3>

##### We'll estimate minimum revenue based on no. of ratings.

##### It assumes that the number of ratings directly correlates with the number of sales.
##### This may not always be true, as not every customer who buys a product leaves a rating.
##### This approach can give an idea of minimum revenue generated.

#### 6a) Revenue across main_categories

In [ ]:

# Calculate revenue
data['revenue'] = data['no_of_ratings'] * data['actual_price']


# Aggregating revenue by main_category
category_revenue = data.groupby('main_category')['revenue'].sum().sort_values(ascending=False)

print(category_revenue)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(15, 6))
sns.barplot(x=category_revenue.index, y=category_revenue.values, palette='magma', width=0.8)
plt.title('Total Revenue by Main Category', fontsize=15)
plt.xlabel('Main Category', fontsize=10)
plt.ylabel('Total Revenue', fontsize=10)
plt.xticks(rotation=90, fontsize=10)
plt.yticks(fontsize=10)
plt.show()


#### 6b) Top 10 products by revenue

In [ ]:
data.to_csv("Amazon_Products_cleaned_dataset.csv",index=False)

In [ ]:

# Assuming 'data' is your DataFrame
top_products = data.groupby(['name']).agg({'revenue':'sum'}).sort_values(by='revenue', ascending=False).head(10).reset_index()

plt.figure(figsize=(12, 6))
bars = sns.barplot(x='name', y='revenue', data=top_products, palette='viridis')
plt.title('Top 10 Products by Revenue')
plt.xlabel('Product Name')
plt.ylabel('Revenue')
plt.xticks(rotation=90)

# Add the exact sales numbers inside the bars
for bar in bars.patches:
    yval = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2.0, 
        yval / 2, 
        f'{yval:,.2f}', 
        ha='center', 
        va='center', 
        color='white',
        rotation='vertical',
        fontsize=10
    )

plt.show()


<h3 style = "background-color: #000033;
             padding: 15px;
             font: bold 42px arial;
             color: #ccebff;
             border: 2px #e6e6ff;
             border-radius: 8px">
Enjoyed the notebook? An upvote would be truly appreciated!</h3>